In [1]:
#default_exp hybrid

In [2]:
#export

import os
import pickle
import attr

import pandas as pd
import scipy
import numpy as np

from game_recommender import steam_data, content_based, user_based

In [3]:
%cd ..

/home/kuba/Projects/game_recommender


In [4]:
game_dataset = steam_data.get_steam_ratings_dataset()

In [5]:
user_based_recommender = user_based.UserBasedRecommender.make_from_steam_ratings('log_hours')

In [6]:
content_based_recommender = content_based.ContentBasedRecommender.make_from_steam_metadata()

In [7]:
steam_df = steam_data.load_steam_df() 

In [8]:
chosen_games_substring = 'counter strike'
chosen_games_df = steam_data.get_games_by_name(steam_df, chosen_games_substring) 

In [9]:
ratings_df = pd.concat([game_dataset.data, game_dataset.target], axis=1)
item_user_pivoted_df = ratings_df.pivot_table(index='name', columns='user_id', values='log_hours')


i = 10
user_id = user_based_recommender.user_similarity_searcher.df.iloc[10,0]
user_ratings_raw = item_user_pivoted_df.iloc[:,10]
user_ratings = user_ratings_raw[~user_ratings_raw.isna()]

In [10]:
user_ratings

name
counter strike                     3.135494
counter strike global offensive    6.194405
counter strike nexon zombies       0.693147
counter strike source              0.693147
day of defeat                      0.693147
deathmatch classic                 0.693147
eldevin                            0.693147
half life                          0.693147
half life blue shift               0.693147
half life opposing force           0.693147
planetside 2                       0.693147
raceroom racing experience         0.336472
realm of the mad god               0.693147
ricochet                           0.693147
team fortress classic              0.693147
unturned                           0.693147
warface                            0.693147
Name: 547685, dtype: float64

In [11]:
content_based_recommender.recommend_similar_games(
    user_ratings.index,
    ratings=user_ratings.values,
    n_recommended=50,
    n_similar=10
)

TypeError: recommend_similar_games() got an unexpected keyword argument 'ratings'

In [ ]:
user_based_recommender.recommend_games_from_similar_users(user_ratings=user_ratings, n_recommended=50, n_similar=10)